In [1]:
from pyDOE import lhs
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io
import time

from desdeo_problem.Problem import DataProblem
from desdeo_problem.surrogatemodels.SurrogateKriging import SurrogateKriging
from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs.ProbRVEA import RVEA
from desdeo_emo.EAs.ProbRVEA import ProbRVEAv1 as ProbRVEA
from desdeo_emo.EAs.ProbRVEA import ProbRVEAv3
from desdeo_emo.EAs.ProbRVEA import HybRVEA
from desdeo_emo.EAs.ProbMOEAD import MOEA_D
from desdeo_emo.EAs.ProbMOEAD import ProbMOEAD
from desdeo_emo.EAs.ProbMOEAD import ProbMOEAD_v3
from desdeo_emo.EAs.ProbMOEAD import HybMOEAD

In [2]:
# Problem setup
nvars = 10
nobjs = 5
nsamples = 109

In [3]:
# Read dataset
folder_data = 'AM_Samples_109_Final'
problem_testbench = 'DDMOPP'
problem_name = 'P1'
sampling = 'LHS'
run = 0

if problem_testbench == "DDMOPP":
    mat = scipy.io.loadmat('./' + folder_data + '/Initial_Population_' + problem_testbench + '_' + sampling +
                        '_AM_' + str(nvars) + '_109.mat')
    x = ((mat['Initial_Population_'+problem_testbench])[0][run])[0]
    mat = scipy.io.loadmat('./'+folder_data+'/Obj_vals_DDMOPP_'+sampling+'_AM_'+problem_name+'_'
                                    + str(nobjs) + '_' + str(nvars) + '_109.mat')
    y = ((mat['Obj_vals_DDMOPP'])[0][run])[0]
else:
    mat = scipy.io.loadmat('./' + folder_data + '/Initial_Population_DTLZ_'+sampling+'_AM_' + str(nvars) + '_109.mat')
    prob = test_problem_builder(
        name=problem_name, n_of_objectives=nobjs, n_of_variables=nvars
    )
    x = ((mat['Initial_Population_DTLZ'])[0][run])[0]
    y = prob.evaluate(x)[0]


In [4]:
# Create problem object
is_data = True
x_names = [f'x{i}' for i in range(1,nvars+1)]
y_names = [f'f{i}' for i in range(1,nobjs+1)]
row_names = ['lower_bound','upper_bound']
if is_data is False:
    prob = test_problem_builder(problem_name, nvars, nobjs)
    x = lhs(nvars, nsamples)
    y = prob.evaluate(x)
    data = pd.DataFrame(np.hstack((x,y.objectives)), columns=x_names+y_names)
else:
    data = pd.DataFrame(np.hstack((x,y)), columns=x_names+y_names)
if problem_testbench == 'DDMOPP':
    x_low = np.ones(nvars)*-1
    x_high = np.ones(nvars)
elif problem_testbench == 'DTLZ':
    x_low = np.ones(nvars)*0
    x_high = np.ones(nvars)    
bounds = pd.DataFrame(np.vstack((x_low,x_high)), columns=x_names, index=row_names)
problem = DataProblem(data=data, variable_names=x_names, objective_names=y_names,bounds=bounds)
start = time.time()
problem.train(SurrogateKriging)
end = time.time()
time_taken = end - start
print("Kriging surrogates built in:",time_taken,"(secs)")

Kriging surrogates built in: 2.092156171798706 (secs)


In [21]:
# Run Generic RVEA
max_func_evals = 400

evolver_opt = RVEA(problem, use_surrogates=True, n_gen_per_iter=100, total_function_evaluations=max_func_evals)
while evolver_opt.continue_evolution():
        evolver_opt.iterate()
        print("FE count:",evolver_opt._function_evaluation_count)

FE count: 404


In [22]:
# Run Probabilstic RVEA
max_func_evals = 400

evolver_opt = ProbRVEA(problem, use_surrogates=True, n_gen_per_iter=100, total_function_evaluations=max_func_evals)
while evolver_opt.continue_evolution():
        evolver_opt.iterate()
        print("FE count:",evolver_opt._function_evaluation_count)

Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/i/desdeo_emo_moead/desdeo_emo/othertools/ProbabilityWrong.py", line 252, in fun_wrapper2
    return self.compute_probability_wrong_fast(*indices)
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/mnt/i/desdeo_emo_moead/desdeo_emo/othertools/ProbabilityWrong.py", line 202, in compute_probability_wrong_fast
    prob_wrong = integrate.quad(prob_mult_vect, 0, np.inf, args=(pdf_A, cdf_B))


KeyboardInterrupt: 

In [24]:
# Run Probabilstic RVEA (fast)
max_func_evals = 400

evolver_opt = ProbRVEAv3(problem, use_surrogates=True, n_gen_per_iter=100, total_function_evaluations=max_func_evals)
while evolver_opt.continue_evolution():
        evolver_opt.iterate()
        print("FE count:",evolver_opt._function_evaluation_count)

FE count: 402


In [ ]:
# Run Hybrid  Probabilstic RVEA
max_func_evals = 400

evolver_opt = HybRVEA(problem, use_surrogates=True, n_gen_per_iter=100, total_function_evaluations=max_func_evals)
while evolver_opt.continue_evolution():
        evolver_opt.iterate()
        print("FE count:",evolver_opt._function_evaluation_count)

In [8]:
# Run Generic MOEA/D
max_func_evals = 1000

evolver_opt = MOEA_D(problem, use_surrogates=True, n_gen_per_iter=100, total_function_evaluations=max_func_evals)
while evolver_opt.continue_evolution():
        evolver_opt.iterate()
        print("FE count:",evolver_opt._function_evaluation_count)

126
FE count: 1008


In [20]:
# Run Probabilstic MOEA/D
max_func_evals = 140

evolver_opt = ProbMOEAD(problem, use_surrogates=True, n_gen_per_iter=100, total_function_evaluations=max_func_evals)
while evolver_opt.continue_evolution():
        evolver_opt.iterate()
        print("FE count:",evolver_opt._function_evaluation_count)



126


KeyboardInterrupt: 

In [9]:
# Run Probabilstic MOEA/D (fast)
max_func_evals = 1000

evolver_opt = ProbMOEAD_v3(problem, use_surrogates=True, n_gen_per_iter=100, total_function_evaluations=max_func_evals)
while evolver_opt.continue_evolution():
        evolver_opt.iterate()
        print("FE count:",evolver_opt._function_evaluation_count)

126
FE count: 1008


In [13]:
objectives = evolver_opt.population.objectives
fig1 = go.Figure(data=go.Scatter3d(x=objectives[:,0],y=objectives[:,1],z=objectives[:,2], mode="markers"))
fig1


In [ ]:
# Run Hybrid Probabilstic MOEA/D
max_func_evals = 400

evolver_opt = HybMOEAD(problem, use_surrogates=True, n_gen_per_iter=100, total_function_evaluations=max_func_evals)
while evolver_opt.continue_evolution():
        evolver_opt.iterate()
        print("FE count:",evolver_opt._function_evaluation_count)